In [1]:
import sagemaker
import joblib
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [36]:
!pip uninstall scikit-learn -y
!pip install scikit-learn=='1.2.1'

Found existing installation: scikit-learn 1.2.1
Uninstalling scikit-learn-1.2.1:
  Successfully uninstalled scikit-learn-1.2.1
  Using cached scikit_learn-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-core 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-features 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-multimodal 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-tabular 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.


In [2]:
session = sagemaker.Session()
bucket = session.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
df = pd.read_csv("train.csv", header=None)

In [4]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [5]:
model = KNeighborsClassifier()

In [6]:
model.fit(X, y)

KNeighborsClassifier()

In [8]:
joblib.dump(model, "model.joblib")

['model.joblib']

In [9]:
!tar -czvf model.tar.gz model.joblib 

model.joblib


In [10]:
model_uri = session.upload_data("model.tar.gz", bucket, "custom_model/heart")

In [45]:
model_uri

's3://sagemaker-us-east-1-381492271173/custom_model/heart/model.tar.gz'

In [11]:
%%writefile entrypoint.py
import joblib
import os
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


Overwriting entrypoint.py


In [12]:
!python -c "from entrypoint import model_fn; model_fn('')"

In [13]:
sklearn_model = SKLearnModel(model_data=model_uri,
                             role="LabRole",
                             entry_point="entrypoint.py",
                             framework_version="1.2-1")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [14]:
predictor = sklearn_model.deploy(initial_instance_count=1,
                                 instance_type='ml.m5.xlarge',
                                 serializer=CSVSerializer(),
                                 deserializer=CSVDeserializer()
                                )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
-----!

In [16]:
predictor.predict('57,0,0,140,241,0,1,123,1,0.2,1,0,3\n45,1,3,110,264,0,1,132,0,1.2,1,0,3\n')

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint sagemaker-scikit-learn-2024-03-08-01-31-12-858 of account 381492271173 not found.